<a href="https://colab.research.google.com/github/BonTapHoa/paic-tho-san-hang-e/blob/main/test50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline: IELTS Writing Task 1 Scoring

Chào mừng quý thí sinh đến với PTNK AI Challenge do The Noders Community tổ chức.

Trong notebook này, chúng ta sẽ xây dựng một mô hình để chấm điểm bài thi IELTS Writing Task 1.

### Mục tiêu
*   **Input**: Bài văn (Essay) của thí sinh.
*   **Output**: Điểm số (Band Score) từ 1.0 - 9.0.
*   **Đánh giá**: MAE (Mean Absolute Error). Giá trị nhỏ hơn biểu thị hiệu năng tốt hơn.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# @title Cài đặt thư viện
# 1. Cài đặt & Import thư viện
# Chúng ta sẽ dùng thư viện Transformers của HuggingFace - tiêu chuẩn vàng trong NLP hiện nay.

import os
import sys
import warnings
import logging

# --- CẤU HÌNH ẨN CÁC WARNING KHÔNG CẦN THIẾT ---
# Tắt log của TensorFlow (nếu có) để giảm thông tin không cần thiết cho người mới
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'
logging.getLogger("transformers").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cài đặt các thư viện cần thiết nếu chưa có
try:
    import transformers
    import datasets
    import gdown
except ImportError:
    print("Đang cài đặt các thư viện: transformers, datasets, accelerate, gdown, scikit-learn...")
    # Thêm -q để cài đặt im lặng hơn
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "transformers", "datasets", "accelerate", "gdown", "scikit-learn"])

import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, TrainingArguments, Trainer
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import EarlyStoppingCallback
from transformers import get_cosine_schedule_with_warmup
from transformers import DataCollatorWithPadding
from transformers import AutoModel, PreTrainedModel, AutoConfig
from transformers.modeling_outputs import SequenceClassifierOutput
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import gc # Garbage Collector để dọn rác bộ nhớ
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.model_selection import StratifiedKFold

try:
    from safetensors.torch import load_file
except ImportError:
    print("Đang cài đặt thư viện safetensors...")
    os.system("pip install safetensors")
    from safetensors.torch import load_file

# Kiểm tra GPU (Nếu có GPU thì train sẽ nhanh hơn rất nhiều)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Thiết bị sử dụng: {device.upper()}")

Thiết bị sử dụng: CUDA


In [4]:
# @title Tải dữ liệu
# 2. Tải dữ liệu từ Google Drive
# Link dữ liệu đã được cung cấp sẵn
train_file_id = '1VLdW_0J7OZdBOPlD-R__JUUCYfGLYpz9'
test_file_id = '1oZioLvy29aLzCZSYALnlYVLSitiQI5Tb'

def download_from_drive(file_id, output_name):
    url = f'https://drive.google.com/uc?id={file_id}'
    if not os.path.exists(output_name):
        print(f"Đang tải {output_name}...")
        gdown.download(url, output_name, quiet=False)
    else:
        print(f"{output_name} đã tồn tại.")

download_from_drive(train_file_id, 'train.csv')
download_from_drive(test_file_id, 'test.csv')

# Đọc dữ liệu
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

print(f"Kích thước tập Train: {df_train.shape}")
print(f"Kích thước tập Test: {df_test.shape}")
display(df_train.head(3))

Đang tải train.csv...


Downloading...
From: https://drive.google.com/uc?id=1VLdW_0J7OZdBOPlD-R__JUUCYfGLYpz9
To: /content/train.csv
100%|██████████| 8.32M/8.32M [00:00<00:00, 44.6MB/s]


Đang tải test.csv...


Downloading...
From: https://drive.google.com/uc?id=1oZioLvy29aLzCZSYALnlYVLSitiQI5Tb
To: /content/test.csv
100%|██████████| 1.64M/1.64M [00:00<00:00, 13.1MB/s]


Kích thước tập Train: (4517, 9)
Kích thước tập Test: (897, 4)


,ID,prompt,image_description,essay,Overall,TR,CC,LR,GRA
0,0,The charts below show the proportions of Briti...,The image shows two pie charts comparing the p...,The presented pie charts display information a...,6.5,7.5,6.0,6.0,6.0
1,1,The plans below show a public park when it fir...,The image displays two maps of Grange Park sho...,The plans illustrate how Grange Park appeared ...,9.0,9.0,9.0,9.0,9.0
2,2,The graph shows the proportion of four differe...,"The image shows a line graph titled ""Recycling...",The line graph illustrates the percentages of ...,8.0,7.0,8.0,7.0,9.0


In [5]:
# @title Hyperparameters
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 0.3
LAYER_DECAY = 0.7
NUM_EPOCHS = 20
BATCH_SIZE = 20
GRADIENT_ACCUMULATION_STEPS = 64 // BATCH_SIZE

N_FOLDS = 5
CURRENT_FOLD = 3
SEED = 42

PATH = '/content/drive/MyDrive/PTNK/PAIC/ielts_12'
CHECKPOINT = [None, None, None, '/content/drive/MyDrive/PTNK/PAIC/ielts_12/fold_3/checkpoint-122', None]

In [6]:
# @title Định nghĩa class model
def freeze_layers(model, num_frozen_layers=0):
    """
    Hàm đóng băng Embeddings và N lớp đầu tiên của Encoder.
    """
    if num_frozen_layers == 0:
        return

    # 1. Đóng băng lớp Embeddings (Luôn nên làm khi dữ liệu ít)
    # Lớp này chứa từ vựng căn bản, không cần thay đổi.
    for param in model.bert.embeddings.parameters():
        param.requires_grad = False

    # 2. Đóng băng các lớp Encoder đầu tiên
    # model.bert.encoder.layer là danh sách các lớp Transformer
    for i in range(num_frozen_layers):
        for param in model.bert.encoder.layer[i].parameters():
            param.requires_grad = False

    print(f"❄️ Đã đóng băng Embeddings và {num_frozen_layers} lớp Encoder đầu tiên.")

def get_optimizer_grouped_parameters(model, learning_rate, weight_decay, layer_decay=0.9):
    """
    Phân nhóm tham số để áp dụng LLRD.
    Layer trên cùng (Classifier) nhận learning_rate gốc.
    Các layer thấp hơn nhận: lr * (layer_decay ^ depth)
    """

    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = []

    # --- 1. NHÓM CLASSIFIER & POOLER (HEAD) ---
    # Nhóm này quan trọng nhất để fix Underfitting -> LR cao nhất
    head_params = []
    for name, param in model.named_parameters():
        if "classifier" in name or "pooler" in name:
            head_params.append((name, param))

    optimizer_grouped_parameters.extend([
        {
            "params": [p for n, p in head_params if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
            "lr": learning_rate,
        },
        {
            "params": [p for n, p in head_params if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
            "lr": learning_rate,
        },
    ])

    # --- 2. NHÓM BACKBONE (DeBERTa Layers) ---
    if hasattr(model, "bert"):
        backbone = model.bert
    else:
        backbone = model.deberta

    num_layers = backbone.config.num_hidden_layers

    for layer_i in range(num_layers - 1, -1, -1):
        layer_name = f"encoder.layer.{layer_i}."
        current_lr = learning_rate * (layer_decay ** (num_layers - layer_i))

        layer_params = []
        for name, param in model.named_parameters():
            if layer_name in name and ("bert" in name or "deberta" in name):
                layer_params.append((name, param))

        optimizer_grouped_parameters.extend([
            {
                "params": [p for n, p in layer_params if not any(nd in n for nd in no_decay)],
                "weight_decay": weight_decay,
                "lr": current_lr,
            },
            {
                "params": [p for n, p in layer_params if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
                "lr": current_lr,
            },
        ])

    # --- 3. NHÓM EMBEDDINGS (Đáy cùng) ---
    embedding_lr = learning_rate * (layer_decay ** (num_layers + 1))
    embedding_params = []
    for name, param in model.named_parameters():
        if "embeddings" in name and ("bert" in name or "deberta" in name):
            embedding_params.append((name, param))

    optimizer_grouped_parameters.extend([
        {
            "params": [p for n, p in embedding_params if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
            "lr": embedding_lr,
        },
        {
            "params": [p for n, p in embedding_params if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
            "lr": embedding_lr,
        },
    ])

    return optimizer_grouped_parameters

class AttentionPooling(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(in_dim, in_dim),
            nn.LayerNorm(in_dim),
            nn.GELU(),
            nn.Linear(in_dim, 1)
        )

    def forward(self, last_hidden_state, attention_mask):
        w = self.attention(last_hidden_state)
        if attention_mask is not None:
            padding_mask = attention_mask.unsqueeze(-1)
            w = w.masked_fill(padding_mask == 0, float('-inf'))
        w = torch.softmax(w, dim=1)
        context_vector = torch.sum(w * last_hidden_state, dim=1)
        return context_vector

class IELTSModel(nn.Module):
    def __init__(self, checkpoint, num_labels=1):
        super(IELTSModel, self).__init__()
        self.bert = AutoModel.from_pretrained(checkpoint)
        hidden_size = self.bert.config.hidden_size
        self.pooler = AttentionPooling(hidden_size)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.LayerNorm(512),
            nn.Dropout(0.5),
            nn.Linear(512, num_labels)
        )

        self.loss_fct = nn.L1Loss()

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        feature_vector = self.pooler(last_hidden_state, attention_mask)
        logits = self.classifier(feature_vector)
        loss = None
        if labels is not None:
            loss = self.loss_fct(logits, labels.float())
        return SequenceClassifierOutput(loss=loss, logits=logits)

# Định nghĩa hàm tính metric MAE
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mae = mean_absolute_error(labels, predictions)
    return {"mae": mae}

# Hàm kiểm tra xem đã giảm được bao nhiêu tham số cần train
def count_trainable_parameters(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    all_params = sum(p.numel() for p in model.parameters())
    print(f"Trainable params: {trainable_params:,} || All params: {all_params:,} || % Trainable: {100 * trainable_params / all_params:.2f}%")

In [ ]:
# @title Training model
# Chuẩn hoá dữ liệu, định nghĩa pretrained model
try:
    num_cores = len(os.sched_getaffinity(0))
except AttributeError:
    num_cores = os.cpu_count() or 2

print(f"🚀 Đang sử dụng tối đa {num_cores} nhân CPU để xử lý dữ liệu.")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=1024)

model_checkpoint = "microsoft/deberta-v3-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
sep = tokenizer.sep_token
df_train['text'] = df_train['prompt'] + sep + df_train['image_description'] + sep + df_train['essay']
df_test['text'] = df_test['prompt'] + sep + df_test['image_description'] + sep + df_test['essay']

df_train = df_train.rename(columns={'TR': 'label1', 'CC': 'label2', 'LR': 'label3', 'GRA': 'label4'})
cols_to_combine = ['label1', 'label2', 'label3', 'label4']
df_train['labels'] = df_train[cols_to_combine].values.tolist()

# Tokenize tập test
test_dataset = datasets.Dataset.from_pandas(df_test[['text']])
tokenized_test = test_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=num_cores,
    remove_columns=["text"] if "text" in test_dataset.column_names else []
)
tokenized_test.set_format("torch", columns=["input_ids", "attention_mask", "token_type_ids"])

# Chia Fold tập train và val
df_train['fold'] = -1
y = df_train['Overall'].astype(str)

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

for fold, (train_idx, val_idx) in enumerate(skf.split(df_train, y)):
    df_train.loc[val_idx, 'fold'] = fold

print("Phân bố số lượng mẫu trong các fold:")
print(df_train['fold'].value_counts())

# Vòng lặp train từng Fold
for fold in range(CURRENT_FOLD, N_FOLDS):
    CHECKPOINT_DIR = None

    # Khởi tạo lại Model MỚI HOÀN TOÀN (để reset trọng số)
    model = IELTSModel(model_checkpoint, num_labels=4)
    model.to(device)

    if CHECKPOINT[fold] is not None:
        CHECKPOINT_DIR = CHECKPOINT[fold]
        weight_path = os.path.join(CHECKPOINT_DIR, "pytorch_model.bin")
        use_safetensors = False

        if not os.path.exists(weight_path):
            weight_path = os.path.join(CHECKPOINT_DIR, "model.safetensors")
            if os.path.exists(weight_path):
                use_safetensors = True

        print(f"Đang load weights từ: {weight_path}")
        try:
            if use_safetensors:
                state_dict = load_file(weight_path, device="cpu")
            else:
                state_dict = torch.load(weight_path, map_location=torch.device('cpu'), weights_only=False)
            model.load_state_dict(state_dict)
            print("✅ Đã load model thành công!")
        except Exception as e:
            print(f"❌ Lỗi khi load weights: {e}")
            if not os.path.exists(weight_path):
                print(f"⚠️ Kiểm tra lại đường dẫn, file không tồn tại: {weight_path}")

        print(f"Đang load tokenizer từ thư mục: {CHECKPOINT_DIR}")
        try:
            tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_DIR)
            print("✅ Đã load tokenizer thành công!")
        except Exception as e:
            print(f"❌ Lỗi tokenizer: {e}")

    fold_output_dir = f"{PATH}/fold_{fold}"

    train_df = df_train[df_train['fold'] != fold].reset_index(drop=True)
    val_df = df_train[df_train['fold'] == fold].reset_index(drop=True)

    train_dataset = datasets.Dataset.from_pandas(train_df[['text', 'labels']])
    val_dataset = datasets.Dataset.from_pandas(val_df[['text', 'labels']])

    try:
        num_cores = len(os.sched_getaffinity(0))
    except AttributeError:
        num_cores = os.cpu_count() or 2

    print(f"🚀 Đang sử dụng tối đa {num_cores} nhân CPU để xử lý dữ liệu.")

    tokenized_train = train_dataset.map(
        tokenize_function,
        batched=True,
        num_proc=num_cores,
        remove_columns=["text"]
    )

    tokenized_val = val_dataset.map(
        tokenize_function,
        batched=True,
        num_proc=num_cores,
        remove_columns=["text"]
    )

    # Set format torch
    tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "token_type_ids", "labels"])
    tokenized_val.set_format("torch", columns=["input_ids", "attention_mask", "token_type_ids", "labels"])

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    optimizer_grouped_parameters = get_optimizer_grouped_parameters(
        model,
        LEARNING_RATE,
        WEIGHT_DECAY,
        LAYER_DECAY
    )

    optimizer = optim.AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE, eps=1e-8)

    num_train_steps = (len(tokenized_train) // BATCH_SIZE) * NUM_EPOCHS

    lr_scheduler = get_cosine_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=int(0.1 * num_train_steps),
        num_training_steps=num_train_steps
    )

    total_layers = model.bert.config.num_hidden_layers

    num_frozen = int(total_layers * 0.4)

    freeze_layers(model, num_frozen_layers=num_frozen)

    count_trainable_parameters(model)

    args = TrainingArguments(
        output_dir=fold_output_dir,
        logging_strategy="epoch",
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=LEARNING_RATE, # Tham số này chỉ để log, thực tế dùng optimizer truyền vào
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        num_train_epochs=NUM_EPOCHS,
        weight_decay=WEIGHT_DECAY,
        load_best_model_at_end=True,
        metric_for_best_model="mae",
        greater_is_better=False,
        save_total_limit=2,
        report_to='none',
        fp16=False,                            # Tắt FP16 truyền thống
        bf16=True,                             # Bật BF16 (Ngon hơn trên A100)
        tf32=True,                             # Bật TensorFloat-32 (Mặc định A100 hỗ trợ, nhưng khai báo cho chắc)
        dataloader_num_workers=num_cores,      # Tăng workers để CPU nạp data kịp cho GPU (Colab thường có 2-4 vCPU)
        dataloader_pin_memory=True,            # Ghim bộ nhớ để chuyển từ RAM -> VRAM nhanh hơn
        remove_unused_columns=True,
        dataloader_prefetch_factor=2,
        group_by_length=True,
        torch_compile=True
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        compute_metrics=compute_metrics,
        data_collator=data_collator,
        optimizers=(optimizer, lr_scheduler),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
    )

    if CHECKPOINT_DIR is not None:
        trainer.train(resume_from_checkpoint=CHECKPOINT_DIR)
    else:
        trainer.train()

    trainer.save_model(fold_output_dir)
    tokenizer.save_pretrained(fold_output_dir)

    del model, trainer, optimizer, lr_scheduler
    torch.cuda.empty_cache()
    gc.collect()

🚀 Đang sử dụng tối đa 12 nhân CPU để xử lý dữ liệu.


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Map (num_proc=12):   0%|          | 0/897 [00:00<?, ? examples/s]

...

Đang load weights từ: /content/drive/MyDrive/PTNK/PAIC/ielts_12/fold_3/checkpoint-122/model.safetensors
✅ Đã load model thành công!
Đang load tokenizer từ thư mục: /content/drive/MyDrive/PTNK/PAIC/ielts_12/fold_3/checkpoint-122
✅ Đã load tokenizer thành công!
🚀 Đang sử dụng tối đa 12 nhân CPU để xử lý dữ liệu.


Map (num_proc=12):   0%|          | 0/3614 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/903 [00:00<?, ? examples/s]

❄️ Đã đóng băng Embeddings và 9 lớp Encoder đầu tiên.
Trainable params: 191,050,245 || All params: 435,592,709 || % Trainable: 43.86%


Epoch,Training Loss,Validation Loss,Mae
3,1.248800,1.021011,1.021011
4,1.050700,0.877617,0.877617
5,0.961400,0.792995,0.792995
6,0.880500,0.742585,0.742585
7,0.827100,0.696083,0.696082
8,0.787000,0.644458,0.644458
9,0.755900,0.629603,0.629603
10,0.738200,0.642087,0.642087
11,0.706600,0.615224,0.615224
12,0.707600,0.613986,0.613986


In [ ]:
# @title predict k-fold
all_fold_preds = []

def pre_processing(arr):
    """
    Hàm xử lý hậu kỳ cho mô hình 4 labels (TR, CC, LR, GRA).
    Input: Mảng numpy kích thước (N, 4) hoặc (N*4,)
    Output: Mảng numpy kích thước (N,) chứa điểm Overall đã làm tròn.
    """
    # 1. Kiểm tra kích thước dữ liệu
    if arr.ndim == 1:
        arr = arr.reshape(-1, 4)
    overall_raw = np.mean(arr, axis=1)
    y_pred = np.round(overall_raw * 2) / 2
    y_pred = np.clip(y_pred, 1.0, 9.0)
    return y_pred

for fold in range(N_FOLDS):
    fold_dir = f"{PATH}/fold_{fold}"
    print(f"\n🔄 [Fold {fold}] Đang load model từ: {fold_dir}")

    # 1. Kiểm tra folder tồn tại
    if not os.path.exists(fold_dir):
        print(f"⚠️ Không tìm thấy thư mục {fold_dir}. Bỏ qua fold này.")
        continue

    # 2. Khởi tạo lại kiến trúc model (Reset mới hoàn toàn)
    model = IELTSModel(model_checkpoint, num_labels=4)

    # 3. Load Weights (Hỗ trợ cả .safetensors và .bin)
    weight_path_safe = os.path.join(fold_dir, "model.safetensors")
    weight_path_bin = os.path.join(fold_dir, "pytorch_model.bin")

    try:
        if os.path.exists(weight_path_safe):
            state_dict = load_file(weight_path_safe, device="cpu")
        elif os.path.exists(weight_path_bin):
            state_dict = torch.load(weight_path_bin, map_location="cpu")
        else:
            print(f"❌ Không tìm thấy file weights trong {fold_dir}")
            continue

        model.load_state_dict(state_dict)
        print(f"✅ Đã load weights thành công cho Fold {fold}")
    except Exception as e:
        print(f"❌ Lỗi khi load weights Fold {fold}: {e}")
        continue

    # Chuyển model sang GPU
    model.to(device)
    model.eval()

    # 4. Tạo Trainer tạm thời chỉ để predict (Gọn nhẹ hơn viết loop tay)
    # Tăng batch size lên gấp đôi khi predict để chạy nhanh hơn (vì không cần lưu gradient)
    infer_args = TrainingArguments(
        output_dir="/tmp",
        per_device_eval_batch_size=BATCH_SIZE * 2,
        report_to="none",
        fp16=True
    )

    trainer_infer = Trainer(
        model=model,
        args=infer_args,
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer)
    )

    # 5. Thực hiện dự đoán
    raw_preds = trainer_infer.predict(tokenized_test).predictions
    all_fold_preds.append(raw_preds)

    # 6. Dọn dẹp bộ nhớ NGAY LẬP TỨC
    del model, trainer_infer, state_dict, raw_preds
    torch.cuda.empty_cache()
    gc.collect()

# --- TÍNH TOÁN KẾT QUẢ CUỐI CÙNG (ENSEMBLE) ---
if len(all_fold_preds) > 0:
    print(f"\n📊 Đang tổng hợp kết quả từ {len(all_fold_preds)} folds...")

    # Chuyển list thành numpy array: (Số_folds, Số_mẫu, 4_labels)
    all_fold_preds = np.array(all_fold_preds)

    # Lấy trung bình cộng (Mean) dọc theo trục Fold (axis 0)
    avg_preds = np.mean(all_fold_preds, axis=0)

    # Xử lý hậu kỳ (Làm tròn, clip điểm)
    final_scores = pre_processing(avg_preds)

    # Tạo DataFrame Submission
    if 'id' in df_test.columns:
        ids = df_test['id']
    else:
        print("⚠️ Không tìm thấy cột 'id', sử dụng index.")
        ids = df_test.index

    submission = pd.DataFrame({
        'id': ids,
        'Overall': final_scores
    })

    # Hiển thị vài dòng đầu
    print("\nKết quả mẫu (5 dòng đầu):")
    display(submission.head())

    # Lưu file
    save_path = f"{PATH}/submission.csv"
    submission.to_csv(save_path, index=False)
    print(f"\n🎉 Đã lưu file kết quả tại: {save_path}")
    print("👉 Hãy tải file này về và nộp bài nhé!")
else:
    print("❌ Không có dự đoán nào được tạo. Vui lòng kiểm tra lại quá trình train.")

In [20]:
del model, trainer_infer, state_dict, raw_preds, final_scores
torch.cuda.empty_cache()
gc.collect()

7537

In [19]:
# @title predict cũ
fold_dir = f"/content/drive/MyDrive/PTNK/PAIC/ielts_12/fold_4/checkpoint-1220"

def pre_processing(arr):
    """
    Hàm xử lý hậu kỳ cho mô hình 4 labels (TR, CC, LR, GRA).
    Input: Mảng numpy kích thước (N, 4) hoặc (N*4,)
    Output: Mảng numpy kích thước (N,) chứa điểm Overall đã làm tròn.
    """
    # 1. Kiểm tra kích thước dữ liệu
    if arr.ndim == 1:
        arr = arr.reshape(-1, 4)
    overall_raw = np.mean(arr, axis=1)
    y_pred = np.round(overall_raw * 2) / 2
    y_pred = np.clip(y_pred, 1.0, 9.0)
    return y_pred

# 1. Kiểm tra folder tồn tại
if not os.path.exists(fold_dir):
    print(f"⚠️ Không tìm thấy thư mục {fold_dir}.")

# 2. Khởi tạo lại kiến trúc model (Reset mới hoàn toàn)
try:
    num_cores = len(os.sched_getaffinity(0))
except AttributeError:
    num_cores = os.cpu_count() or 2

print(f"🚀 Đang sử dụng tối đa {num_cores} nhân CPU để xử lý dữ liệu.")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=1024)

model_checkpoint = "microsoft/deberta-v3-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
sep = tokenizer.sep_token
df_train['text'] = df_train['prompt'] + sep + df_train['image_description'] + sep + df_train['essay']
df_test['text'] = df_test['prompt'] + sep + df_test['image_description'] + sep + df_test['essay']

df_train = df_train.rename(columns={'TR': 'label1', 'CC': 'label2', 'LR': 'label3', 'GRA': 'label4'})
cols_to_combine = ['label1', 'label2', 'label3', 'label4']
df_train['labels'] = df_train[cols_to_combine].values.tolist()

# Tokenize tập test
test_dataset = datasets.Dataset.from_pandas(df_test[['text']])
tokenized_test = test_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=num_cores,
    remove_columns=["text"] if "text" in test_dataset.column_names else []
)
tokenized_test.set_format("torch", columns=["input_ids", "attention_mask", "token_type_ids"])
model = IELTSModel(model_checkpoint, num_labels=4)

# 3. Load Weights (Hỗ trợ cả .safetensors và .bin)
weight_path_safe = os.path.join(fold_dir, "model.safetensors")
weight_path_bin = os.path.join(fold_dir, "pytorch_model.bin")

try:
    if os.path.exists(weight_path_safe):
        state_dict = load_file(weight_path_safe, device="cpu")
    elif os.path.exists(weight_path_bin):
        state_dict = torch.load(weight_path_bin, map_location="cpu")
    else:
        print(f"❌ Không tìm thấy file weights trong {fold_dir}")

    model.load_state_dict(state_dict)
    print(f"✅ Đã load weights thành công")
except Exception as e:
    print(f"❌ Lỗi khi load weights: {e}")

# Chuyển model sang GPU
model.to(device)
model.eval()

# 4. Tạo Trainer tạm thời chỉ để predict (Gọn nhẹ hơn viết loop tay)
# Tăng batch size lên gấp đôi khi predict để chạy nhanh hơn (vì không cần lưu gradient)
infer_args = TrainingArguments(
    output_dir="/tmp",
    per_device_eval_batch_size=BATCH_SIZE * 2,
    report_to="none",
    fp16=True
)

trainer_infer = Trainer(
    model=model,
    args=infer_args,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer)
)

# 5. Thực hiện dự đoán
raw_preds = trainer_infer.predict(tokenized_test).predictions

# Xử lý hậu kỳ (Làm tròn, clip điểm)
final_scores = pre_processing(raw_preds)

# Tạo DataFrame Submission
if 'id' in df_test.columns:
    ids = df_test['id']
else:
    print("⚠️ Không tìm thấy cột 'id', sử dụng index.")
    ids = df_test.index

submission = pd.DataFrame({
    'id': ids,
    'Overall': final_scores
})

# Hiển thị vài dòng đầu
print("\nKết quả mẫu (5 dòng đầu):")
display(submission.head())

# Lưu file
save_path = f"{fold_dir}/submission.csv"
submission.to_csv(save_path, index=False)
print(f"\n🎉 Đã lưu file kết quả tại: {save_path}")
print("👉 Hãy tải file này về và nộp bài nhé!")

🚀 Đang sử dụng tối đa 2 nhân CPU để xử lý dữ liệu.


Map (num_proc=2):   0%|          | 0/897 [00:00<?, ? examples/s]

✅ Đã load weights thành công


⚠️ Không tìm thấy cột 'id', sử dụng index.

Kết quả mẫu (5 dòng đầu):


,id,Overall
0,0,4.5
1,1,8.5
2,2,6.5
3,3,7.0
4,4,7.0



🎉 Đã lưu file kết quả tại: /content/drive/MyDrive/PTNK/PAIC/ielts_12/fold_4/checkpoint-1220/submission.csv
👉 Hãy tải file này về và nộp bài nhé!


In [29]:
PATH = "/content/drive/MyDrive/PTNK/PAIC/ielts_12"
paths = [
    PATH + '/fold_0/checkpoint-1220',
    PATH + '/fold_1/checkpoint-1159',
    PATH + '/fold_2/checkpoint-793',
    PATH + '/fold_3/checkpoint-1098',
    PATH + '/fold_4/checkpoint-1098',
]

In [27]:
final_df = pd.DataFrame()

# 1. Gom kết quả từ 5 models (folds)
for (idx, path) in enumerate(paths):
    df = pd.read_csv(path + '/submission.csv')
    # Đặt tên cột rõ ràng để tránh nhầm lẫn khi concat
    series = df['Overall'].rename(f'Overall_{idx}')
    final_df = pd.concat([final_df, series], axis=1)

# 2. Tính trung bình cộng (Raw score)
# Kết quả lúc này sẽ là số lẻ (VD: 6.8, 7.1...)
avg_score = final_df.mean(axis=1)

# 3. XỬ LÝ HẬU KỲ (PRE-PROCESSING) CHO KẾT QUẢ CUỐI CÙNG
# Logic: Nhân 2 -> Làm tròn số nguyên -> Chia 2 (để ra steps 0.5)
final_df['Overall'] = np.round(avg_score * 2) / 2

# Clip để đảm bảo điểm nằm trong range 1.0 - 9.0
final_df['Overall'] = np.clip(final_df['Overall'], 1.0, 9.0)

# 4. Tạo dataframe kết quả (Submission)
output_df = pd.DataFrame()
output_df['id'] = df_test['ID'] # Lấy ID từ tập test gốc
output_df['Overall'] = final_df['Overall']

# Lưu và hiển thị
output_df.to_csv(PATH + '/submission.csv', index=False)
output_df.head()

,id,Overall
0,0,4.5
1,1,8.5
2,2,6.5
3,3,7.0
4,4,7.0


In [30]:
final_df = pd.DataFrame()

# 1. Gom kết quả từ 5 models (folds)
for (idx, path) in enumerate(paths):
    df = pd.read_csv(path + '/submission.csv')
    final_df[f'Overall_{idx}'] = df['Overall']

# 2. Tính Max Count (Mode/Voting)
# mode(axis=1) trả về DataFrame các giá trị xuất hiện nhiều nhất theo hàng
# Cột [0] chứa mode đầu tiên tìm thấy.
# Lưu ý: Nếu có trường hợp hòa (VD: 2 cái 6.5, 2 cái 7.0), pandas mặc định trả về giá trị nhỏ hơn ở cột [0].
# Nếu bạn muốn ưu tiên điểm cao khi hòa, hãy thay dòng dưới bằng: final_df['Overall'] = final_df.mode(axis=1).max(axis=1)
final_df['Overall'] = final_df.mode(axis=1)[0]

# 3. Tạo dataframe kết quả
output_df = pd.DataFrame()
output_df['id'] = df_test['ID'] # Lấy ID từ tập test gốc
output_df['Overall'] = final_df['Overall']

# Lưu và hiển thị
output_df.to_csv(PATH + '/submission.csv', index=False)
output_df.head()

,id,Overall
0,0,4.5
1,1,8.5
2,2,6.5
3,3,7.0
4,4,7.0


In [ ]:
# @title predict sieu cũ
def pre_processing(arr):
    """
    Hàm xử lý hậu kỳ cho mô hình 4 labels (TR, CC, LR, GRA).
    Input: Mảng numpy kích thước (N, 4) hoặc (N*4,)
    Output: Mảng numpy kích thước (N,) chứa điểm Overall đã làm tròn.
    """
    # 1. Kiểm tra kích thước dữ liệu
    if arr.ndim == 1:
        arr = arr.reshape(-1, 4)
    overall_raw = np.mean(arr, axis=1)
    y_pred = np.round(overall_raw * 2) / 2
    y_pred = np.clip(y_pred, 1.0, 9.0)
    return y_pred

# 6. Dự đoán trên tập Test & Tạo Submission
predictions = trainer.predict(tokenized_test)
y_pred = pre_processing(predictions.predictions)
if 'id' in df_test.columns:
    ids = df_test['id']
else:
    print("Không tìm thấy cột 'id' trong file test. Sử dụng index làm id...")
    ids = df_test.index

submission = pd.DataFrame({
    'id': ids,
    'Overall': y_pred
})

# Hiển thị vài mẫu để kiểm tra
print(submission.head())

# Lưu file csv
submission.to_csv(PATH + '/submission.csv', index=False)
print("Đã lưu file submission.csv. Vui lòng nộp file này để kiểm tra kết quả.")

# LOAD MODEL

In [ ]:
# @title
import torch
import torch.nn as nn
import os
from transformers import AutoModel, AutoTokenizer
from transformers.modeling_outputs import SequenceClassifierOutput
try:
    from safetensors.torch import load_file
except ImportError:
    print("Đang cài đặt thư viện safetensors...")
    os.system("pip install safetensors")
    from safetensors.torch import load_file

# --- PHẦN 1: ĐỊNH NGHĨA MODEL (Giữ nguyên) ---
class AttentionPooling(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(in_dim, in_dim),
            nn.LayerNorm(in_dim),
            nn.GELU(),
            nn.Linear(in_dim, 1)
        )

    def forward(self, last_hidden_state, attention_mask):
        w = self.attention(last_hidden_state)
        if attention_mask is not None:
            padding_mask = attention_mask.unsqueeze(-1)
            w = w.masked_fill(padding_mask == 0, float('-inf'))
        w = torch.softmax(w, dim=1)
        context_vector = torch.sum(w * last_hidden_state, dim=1)
        return context_vector

class IELTSModel(nn.Module):
    def __init__(self, checkpoint, num_labels=1):
        super(IELTSModel, self).__init__()
        self.bert = AutoModel.from_pretrained(checkpoint)
        hidden_size = self.bert.config.hidden_size
        self.pooler = AttentionPooling(hidden_size)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.LayerNorm(512),
            nn.Dropout(0.1),
            nn.Linear(512, num_labels)
        )
        self.loss_fct = nn.L1Loss()

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        feature_vector = self.pooler(last_hidden_state, attention_mask)
        logits = self.classifier(feature_vector)
        loss = None
        if labels is not None:
            loss = self.loss_fct(logits, labels.float())
        return SequenceClassifierOutput(loss=loss, logits=logits)

# --- PHẦN 2: KHỞI TẠO VÀ LOAD WEIGHTS ---

# Định nghĩa đường dẫn thư mục gốc chứa checkpoint
CHECKPOINT_DIR = "/content/drive/MyDrive/PTNK/PAIC/ielts_9/checkpoint-1220"

# 1. Khởi tạo kiến trúc
print("Đang khởi tạo kiến trúc model...")
model_checkpoint = "microsoft/deberta-v3-large"
model = IELTSModel(model_checkpoint, num_labels=4)

# 2. Load trọng số (Cập nhật logic nhận diện file)
weight_path = os.path.join(CHECKPOINT_DIR, "pytorch_model.bin")
use_safetensors = False

if not os.path.exists(weight_path):
    weight_path = os.path.join(CHECKPOINT_DIR, "model.safetensors")
    if os.path.exists(weight_path):
        use_safetensors = True

print(f"Đang load weights từ: {weight_path}")

try:
    if use_safetensors:
        # Dùng thư viện safetensors để load
        state_dict = load_file(weight_path, device="cpu")
    else:
        # Dùng torch để load file bin truyền thống
        state_dict = torch.load(weight_path, map_location=torch.device('cpu'), weights_only=False)

    model.load_state_dict(state_dict)
    print("✅ Đã load model thành công!")
except Exception as e:
    print(f"❌ Lỗi khi load weights: {e}")
    # Gợi ý debug nếu vẫn lỗi
    if not os.path.exists(weight_path):
        print(f"⚠️ Kiểm tra lại đường dẫn, file không tồn tại: {weight_path}")

# 3. Chuyển sang chế độ dự đoán
model.eval()

# --- PHẦN 3: LOAD TOKENIZER ---
print(f"Đang load tokenizer từ thư mục: {CHECKPOINT_DIR}")
try:
    tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_DIR)
    print("✅ Đã load tokenizer thành công!")
except Exception as e:
    print(f"❌ Lỗi tokenizer: {e}")

In [ ]:
# @title
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

# 1. Hàm xử lý hậu kỳ (Giữ nguyên của bạn)
def pre_processing(arr):
    """
    Hàm xử lý hậu kỳ cho mô hình 4 labels (TR, CC, LR, GRA).
    Input: Mảng numpy kích thước (N, 4) hoặc (N*4,)
    Output: Mảng numpy kích thước (N,) chứa điểm Overall đã làm tròn.
    """
    if arr.ndim == 1:
        arr = arr.reshape(-1, 4)
    overall_raw = np.mean(arr, axis=1)
    y_pred = np.round(overall_raw * 2) / 2
    y_pred = np.clip(y_pred, 1.0, 9.0)
    return y_pred

# 2. Hàm dự đoán thủ công (Thay thế cho trainer.predict)
from transformers import DataCollatorWithPadding

# Đảm bảo data_collator đã được khởi tạo (nếu chưa chạy cell trước đó)
if 'data_collator' not in globals():
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def predict_manual(model, tokenized_dataset, batch_size=16):
    # --- SỬA LỖI TẠI ĐÂY: Thêm tham số collate_fn ---
    dataloader = DataLoader(
        tokenized_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=data_collator  # <--- Quan trọng: Giúp padding dữ liệu trong batch
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    all_logits = []

    print("Đang chạy dự đoán...")
    with torch.no_grad():
        for batch in tqdm(dataloader):
            # Chuyển dữ liệu sang thiết bị
            inputs = {k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask', 'token_type_ids']}

            outputs = model(**inputs)
            logits = outputs.logits
            all_logits.append(logits.cpu().numpy())

    return np.concatenate(all_logits, axis=0)

# --- THỰC THI ---

# Kiểm tra xem biến tokenized_test và df_test có tồn tại không
# (Bạn phải chạy các cell xử lý dữ liệu phía trên Notebook trước cell này)
try:
    # 6. Dự đoán trên tập Test
    raw_predictions = predict_manual(model, tokenized_test)

    # Xử lý hậu kỳ
    y_pred = pre_processing(raw_predictions)

    # Tạo DataFrame Submission
    if 'id' in df_test.columns:
        ids = df_test['id']
    else:
        print("Không tìm thấy cột 'id' trong file test. Sử dụng index làm id...")
        ids = df_test.index

    submission = pd.DataFrame({
        'id': ids,
        'Overall': y_pred
    })

    # Hiển thị và lưu file
    print(submission.head())
    submission.to_csv(CHECKPOINT_DIR + '/submission.csv', index=False)
    print("\n✅ Đã lưu file submission.csv thành công!")

except NameError as e:
    print(f"❌ Lỗi thiếu biến dữ liệu: {e}")
    print("👉 Bạn cần chạy lại các cell 'Load dữ liệu' và 'Tokenization' (Cell 2, 3, 4 trong notebook gốc) để tạo lại biến 'df_test' và 'tokenized_test'.")

# LOAD MODEL TRAIN TIẾP

In [ ]:
# @title
import torch
import torch.optim as optim
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import get_cosine_schedule_with_warmup, EarlyStoppingCallback
import torch
import torch.nn as nn
import os
from transformers import AutoModel, AutoTokenizer
from transformers.modeling_outputs import SequenceClassifierOutput
try:
    from safetensors.torch import load_file
except ImportError:
    print("Đang cài đặt thư viện safetensors...")
    os.system("pip install safetensors")
    from safetensors.torch import load_file

# --- 1. CẤU HÌNH LẠI CÁC THAM SỐ (Phải giống hệt lúc train cũ) ---
# Đường dẫn đến checkpoint bạn đang có trên Drive
CHECKPOINT_DIR = "/content/drive/MyDrive/PTNK/PAIC/ielts_7/checkpoint-1254"

# Các tham số Hyperparameters (Lấy từ notebook gốc)
learning_rate = 5e-5
weight_decay = 0.1
layer_decay = 0.8
batch_size = 16
num_epochs = 40
gradient_accumulation_steps = 64 // batch_size

# --- 2. TẠO LẠI OPTIMIZER & SCHEDULER (Vì notebook dùng LLRD custom) ---
# Hàm phân nhóm tham số (LLRD) - Copy y chang từ notebook gốc
def get_optimizer_grouped_parameters(model, learning_rate, weight_decay, layer_decay=0.9):
    """
    Phân nhóm tham số để áp dụng LLRD.
    Layer trên cùng (Classifier) nhận learning_rate gốc.
    Các layer thấp hơn nhận: lr * (layer_decay ^ depth)
    """

    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = []

    # --- 1. NHÓM CLASSIFIER & POOLER (HEAD) ---
    # Nhóm này quan trọng nhất để fix Underfitting -> LR cao nhất
    head_params = []
    for name, param in model.named_parameters():
        if "classifier" in name or "pooler" in name:
            head_params.append((name, param))

    optimizer_grouped_parameters.extend([
        {
            "params": [p for n, p in head_params if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
            "lr": learning_rate,
        },
        {
            "params": [p for n, p in head_params if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
            "lr": learning_rate,
        },
    ])

    # --- 2. NHÓM BACKBONE (DeBERTa Layers) ---
    # DeBERTa V3 base có 12 lớp
    if hasattr(model, "bert"):
        backbone = model.bert
    else:
        # Fallback nếu bạn đặt tên biến khác trong class IELTSModel
        backbone = model.deberta

    num_layers = backbone.config.num_hidden_layers

    for layer_i in range(num_layers - 1, -1, -1):
        layer_name = f"encoder.layer.{layer_i}."

        # Công thức decay: LR lớp i = LR gốc * (decay ^ (khoảng cách tới đỉnh))
        current_lr = learning_rate * (layer_decay ** (num_layers - layer_i))

        layer_params = []
        for name, param in model.named_parameters():
            if layer_name in name and ("bert" in name or "deberta" in name):
                layer_params.append((name, param))

        optimizer_grouped_parameters.extend([
            {
                "params": [p for n, p in layer_params if not any(nd in n for nd in no_decay)],
                "weight_decay": weight_decay,
                "lr": current_lr,
            },
            {
                "params": [p for n, p in layer_params if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
                "lr": current_lr,
            },
        ])

    # --- 3. NHÓM EMBEDDINGS (Đáy cùng) ---
    embedding_lr = learning_rate * (layer_decay ** (num_layers + 1))
    embedding_params = []
    for name, param in model.named_parameters():
        if "embeddings" in name and ("bert" in name or "deberta" in name):
            embedding_params.append((name, param))

    optimizer_grouped_parameters.extend([
        {
            "params": [p for n, p in embedding_params if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
            "lr": embedding_lr,
        },
        {
            "params": [p for n, p in embedding_params if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
            "lr": embedding_lr,
        },
    ])

    return optimizer_grouped_parameters

class AttentionPooling(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(in_dim, in_dim),
            nn.LayerNorm(in_dim),
            nn.GELU(),
            nn.Linear(in_dim, 1)
        )

    def forward(self, last_hidden_state, attention_mask):
        w = self.attention(last_hidden_state)
        if attention_mask is not None:
            padding_mask = attention_mask.unsqueeze(-1)
            w = w.masked_fill(padding_mask == 0, float('-inf'))
        w = torch.softmax(w, dim=1)
        context_vector = torch.sum(w * last_hidden_state, dim=1)
        return context_vector

class IELTSModel(nn.Module):
    def __init__(self, checkpoint, num_labels=1):
        super(IELTSModel, self).__init__()
        self.bert = AutoModel.from_pretrained(checkpoint)
        hidden_size = self.bert.config.hidden_size
        self.pooler = AttentionPooling(hidden_size)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.LayerNorm(512),
            nn.Dropout(0.5),
            nn.Linear(512, num_labels)
        )

        self.loss_fct = nn.L1Loss()

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        feature_vector = self.pooler(last_hidden_state, attention_mask)
        logits = self.classifier(feature_vector)
        loss = None
        if labels is not None:
            loss = self.loss_fct(logits, labels.float())
        return SequenceClassifierOutput(loss=loss, logits=logits)

model_checkpoint = "microsoft/deberta-v3-large"
model = IELTSModel(model_checkpoint, num_labels=4)
model.to(device)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 1. Khởi tạo kiến trúc
print("Đang khởi tạo kiến trúc model...")

# 2. Load trọng số (Cập nhật logic nhận diện file)
weight_path = os.path.join(CHECKPOINT_DIR, "pytorch_model.bin")
use_safetensors = False

if not os.path.exists(weight_path):
    weight_path = os.path.join(CHECKPOINT_DIR, "model.safetensors")
    if os.path.exists(weight_path):
        use_safetensors = True

print(f"Đang load weights từ: {weight_path}")

try:
    if use_safetensors:
        # Dùng thư viện safetensors để load
        state_dict = load_file(weight_path, device="cpu")
    else:
        # Dùng torch để load file bin truyền thống
        state_dict = torch.load(weight_path, map_location=torch.device('cpu'), weights_only=False)

    model.load_state_dict(state_dict)
    print("✅ Đã load model thành công!")
except Exception as e:
    print(f"❌ Lỗi khi load weights: {e}")
    # Gợi ý debug nếu vẫn lỗi
    if not os.path.exists(weight_path):
        print(f"⚠️ Kiểm tra lại đường dẫn, file không tồn tại: {weight_path}")

# --- PHẦN 3: LOAD TOKENIZER ---
print(f"Đang load tokenizer từ thư mục: {CHECKPOINT_DIR}")
try:
    tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_DIR)
    print("✅ Đã load tokenizer thành công!")
except Exception as e:
    print(f"❌ Lỗi tokenizer: {e}")

# Định nghĩa hàm tính metric MAE
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mae = mean_absolute_error(labels, predictions)
    return {"mae": mae}

optimizer_grouped_parameters = get_optimizer_grouped_parameters(
    model,
    learning_rate,
    weight_decay,
    layer_decay
)

optimizer = optim.AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=1e-8)
# Tổng số bước = (Số mẫu train / batch_size) * số epochs
num_train_steps = (len(tokenized_train) // batch_size) * num_epochs

lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * num_train_steps), # 10% warmup
    num_training_steps=num_train_steps
)

# 5. Cấu hình Arguments
args = TrainingArguments(
    output_dir=PATH,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=learning_rate, # Tham số này chỉ để log, thực tế dùng optimizer truyền vào
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=num_epochs,
    weight_decay=weight_decay,
    load_best_model_at_end=True,
    metric_for_best_model="mae",
    greater_is_better=False,
    save_total_limit=2,
    report_to='none',
    fp16=True,
)

# 6. Khởi tạo Trainer với Optimizer & Scheduler tự tạo
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    optimizers=(optimizer, lr_scheduler),
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

# --- 4. BẮT ĐẦU RESUME ---
print(f"Đang tiếp tục train từ checkpoint: {CHECKPOINT_DIR}")
# resume_from_checkpoint=True sẽ tìm checkpoint mới nhất trong output_dir
# Hoặc truyền đường dẫn cụ thể để chắc chắn
try:
    trainer.train(resume_from_checkpoint=CHECKPOINT_DIR)
    print("Huấn luyện tiếp hoàn tất!")
except ValueError as e:
    print(f"Lỗi resume: {e}")
    print("Gợi ý: Kiểm tra xem trong folder checkpoint có file 'optimizer.pt' và 'scheduler.pt' chưa?")
    print("Nếu không có, hãy set `resume_from_checkpoint=False` để train lại từ đầu với weights hiện tại (nhưng optimizer sẽ bị reset).")